In [1]:
from qmenta.core.platform import Auth, post, parse_response
from getpass import getpass
import pandas as pd
import numpy as np
import registry
import forms
import datetime

## Registries

In [2]:
project_id_reg = 3202

# base url to connect to the central platform
base_url = "https://platform.qmenta.com"
# PUT YOUR USERNAME (EMAIL) HERE
username = "edward.debrouwer@esat.kuleuven.be"
# you will be asked for your password here
password = getpass()

# creation of authentication object
auth_obj = Auth.login(username, password, base_url)

# method to fetch the subjects data
def get_subjects_data(project_id):
    r = post(auth_obj, "/patient_manager/get_patient_list",
            {"_pid":project_id},
            timeout=600.0)

    data_trans = [{
            "id": record["_id"],
            "secret_name":record["patient_secret_name"],
            **{
                k[3:]:record[k]
                for k in record
                if k[:3] == "md_"
            }
    } for record in parse_response(r)]

    for r in data_trans:
        for k in r:
            if isinstance(r[k], dict):
                r[k] = datetime.datetime.fromtimestamp(r[k]["$date"]/1000.0)

    return data_trans

data_reg = get_subjects_data(project_id_reg)
df = pd.DataFrame(data_reg)



········


### Enhance

In [3]:
df_e = registry.enhance_registry_data(df)
df_e.head()

,id,secret_name,age_years,com_cardiovascular_disease,com_chronic_kidney_disease,com_diabetes,com_hypertension,com_lung_disease,com_malignancy,covid19_admission_hospital,...,bmi_in_cat,bmi_in_cat2,Age_in_cat,year_onset,year_reporting,disease_duration,disease_duration_in_cat,disease_duration_in_cat2,ms_type2,current_or_former_smoker
0,1021603.0,COV0_1,35.0,None,None,None,None,None,None,no,...,None,None,1,2004.0,2020.0,16.0,2,1,relapsing-remitting,None
1,1021612.0,COV0_10,43.0,None,None,None,None,None,None,no,...,None,None,1,2004.0,2020.0,16.0,2,1,relapsing-remitting,None
2,1021613.0,COV0_11,31.0,None,None,None,None,None,None,no,...,None,None,1,2020.0,2020.0,0.0,None,None,relapsing-remitting,None
3,1021614.0,COV0_12,64.0,None,None,None,None,None,None,yes,...,None,None,2,1994.0,2020.0,26.0,2,1,progressive MS,yes
4,1021615.0,COV0_13,31.0,None,None,None,None,None,None,no,...,None,None,1,2017.0,2020.0,3.0,1,0,relapsing-remitting,None


## Forms

In [4]:
project_id_forms = 3150

# base url to connect to the central platform
base_url = "https://platform.qmenta.com"
# PUT YOUR USERNAME (EMAIL) HERE
username = "edward.debrouwer@esat.kuleuven.be"
# you will be asked for your password here
password = getpass()

# creation of authentication object
auth_obj = Auth.login(username, password, base_url)

# method to fetch the subjects data
def get_subjects_data(project_id):
    r = post(auth_obj, "/patient_manager/get_patient_list",
            {"_pid":project_id},
            timeout=600.0)

    data_trans = [{
            "id": record["_id"],
            "secret_name":record["patient_secret_name"],
            **{
                k[3:]:record[k]
                for k in record
                if k[:3] == "md_"
            }
    } for record in parse_response(r)]

    for r in data_trans:
        for k in r:
            if isinstance(r[k], dict):
                if "$date" in r[k]:
                    r[k] = datetime.datetime.fromtimestamp(r[k]["$date"]/1000.0)
                elif "$oid" in r[k]:
                    r[k] = r[k]["$oid"]

    return data_trans

data_forms = get_subjects_data(project_id_forms)
df = pd.DataFrame(data_forms)

········


In [5]:
df_e = forms.enhance_forms_data(df)
df_e.head()

,id,secret_name,covid19_date_reporting,covid19_has_symptoms,covid19_sympt_fever,covid19_sympt_dry_cough,covid19_sympt_fatigue,covid19_sympt_pain,covid19_sympt_sore_throat,covid19_sympt_shortness_breath,...,bmi_in_cat,bmi_in_cat2,Age_in_cat,year_onset,year_reporting,disease_duration,disease_duration_in_cat,disease_duration_in_cat2,ms_type2,current_or_former_smoker
0,1013424.0,C_1001,2020-04-24 02:00:00,no,,,,,,,...,None,None,1,2019.0,2020.0,1.0,0,0,relapsing-remitting,None
1,1013427.0,C_1004,2020-05-04 02:00:00,no,,,,,,,...,None,None,2,2005.0,2020.0,15.0,2,1,relapsing-remitting,None
2,1013531.0,C_1005,2020-04-17 02:00:00,,,,,,,,...,None,None,1,NaN,2020.0,NaN,None,None,relapsing-remitting,None
3,1013532.0,C_1006,2020-04-03 02:00:00,,,,,,,,...,None,None,2,NaN,2020.0,NaN,None,None,relapsing-remitting,None
4,1013533.0,C_1007,2020-04-29 02:00:00,yes,no,,,,,yes,...,overweight,overweight,1,NaN,2020.0,NaN,None,None,relapsing-remitting,None
